# __Homework:__ Data talks club data engineering zoomcamp Data loading workshop
Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.
Here are the exercises we will do

## 1. Use a generator
Remember the concept of generator? Let's practice using them to futher our understanding of how they work.
Let's define a generator and then run it as practice.

__Answer the following questions:__

_(I suggest practicing these questions without GPT as the purpose is to further your learning.)_

#### Q1 - What is the sum of the outputs of the generator for limit = 5?

In [5]:
import dlt

def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

sum_ = 0.0

for sqrt_value in generator:
    sum_ += sqrt_value
    
print(sum_)

8.382332347441762


In [ ]:
#### Q2 - What is the 13th number yielded?

In [6]:
limit = 13
generator = square_root_generator(limit)

i = 0
for sqrt_value in generator:
    i += 1
    if limit == i
        print(f'{i}: {sqrt_value}'')

SyntaxError: expected ':' (1523722676.py, line 7)